In [1]:
from transformers import AutoTokenizer

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
tokenizer = AutoTokenizer.from_pretrained("ikit-claw-nlp/toy-llm")

tokenizer_config.json:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/655k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/170 [00:00<?, ?B/s]

In [ ]:
token_ids = tokenizer("<s>Hello world</s>")
print(
    tokenizer.convert_ids_to_tokens(token_ids)
)

3
